In [1146]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix


In [1147]:
dom1 = []
dom2 = []
test_set = []
with open('data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("data/sample.csv")

with open('data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [1148]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [1149]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2149


In [1150]:
data = pd.concat([dom1, dom2], ignore_index=True)

In [1151]:
from sklearn.model_selection import train_test_split
data['model'].fillna(8, inplace=True)


X = data[['text','model']] 
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=data[['model', 'label']], random_state=42)

In [1152]:
y_train

9936     0
25220    0
24497    0
9566     1
18884    0
        ..
29451    0
4594     1
29348    0
26916    0
28576    0
Name: label, Length: 27519, dtype: int64

In [1153]:
print(y_train.value_counts(normalize=True))
print(y_test.value_counts(normalize=True))

print(X_train['model'].value_counts(normalize=True))
print(X_test['model'].value_counts(normalize=True))

label
0    0.654094
1    0.345906
Name: proportion, dtype: float64
label
0    0.65407
1    0.34593
Name: proportion, dtype: float64
model
8.0    0.629347
0.0    0.068716
1.0    0.068534
3.0    0.068534
2.0    0.067989
6.0    0.051274
4.0    0.022930
5.0    0.022675
Name: proportion, dtype: float64
model
8.0    0.629360
0.0    0.068750
3.0    0.068605
1.0    0.068459
2.0    0.068023
6.0    0.051163
4.0    0.022965
5.0    0.022674
Name: proportion, dtype: float64


In [1154]:
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np


X = X_train['text'].tolist()
X1 =X_test['text'].tolist()
max_length = max([len(seq) for seq in X])
max_length1 = max([len(seq) for seq in X1])
word2vec = Word2Vec(X, vector_size=300, window=10, min_count=1, workers=4)
word2vec1 = Word2Vec(X1, vector_size=300, window=10, min_count=1, workers=4)

def get_text_vector(text, word2vec_model):
    vector_list = [word2vec_model.wv[word] for word in text if word in word2vec_model.wv.index_to_key]
    if len(vector_list) == 0:
        return np.zeros(word2vec.vector_size)
    return np.mean(vector_list, axis=0)

X_train = np.array([get_text_vector(text, word2vec) for text in X])
X_test = np.array([get_text_vector(text, word2vec1) for text in X1])

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from gensim.models import Word2Vec
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks





smote = SMOTE(sampling_strategy=0.7)
tomek = TomekLinks(sampling_strategy='majority')


pipeline = Pipeline(steps=[('o', smote), ('u', tomek)])


X_train, y_train = pipeline.fit_resample(X_train, y_train)





In [1155]:
print(y_train.value_counts(normalize=True))

label
0    0.585908
1    0.414092
Name: proportion, dtype: float64


In [1156]:
y_train

0        0
1        0
2        0
3        1
4        0
        ..
30423    1
30424    1
30425    1
30426    1
30427    1
Name: label, Length: 30428, dtype: int64

In [1157]:
print(len(X_train))
print(len(y_train))


30428
30428


In [1158]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to('cuda' if torch.cuda.is_available() else 'cpu')


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

batch_size = 64  
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)




In [1159]:
X_train_tensor

tensor([[ 0.3184,  0.1813, -0.2902,  ..., -0.1018, -0.0891,  0.0371],
        [-0.3175,  0.2098, -0.1292,  ...,  0.0512, -0.3820,  0.1591],
        [-0.2872,  0.2351, -0.0929,  ..., -0.1217, -0.1570,  0.1076],
        ...,
        [-0.2398,  0.2525, -0.0054,  ..., -0.0459, -0.5334,  0.2688],
        [-0.2144,  0.0092, -0.2064,  ...,  0.0857, -0.2025,  0.0526],
        [-0.1500,  0.3700,  0.0022,  ..., -0.0974, -0.0920, -0.0395]])

In [1160]:
y_train_tensor

tensor([0, 0, 0,  ..., 1, 1, 1])

In [1161]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)



torch.Size([30428, 300])
torch.Size([30428])


In [1162]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import TensorDataset, DataLoader

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64  
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


In [1163]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.5):
        super(RNNModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, num_layers=2)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output, (hidden, cell) = self.lstm(x)
        hidden = self.dropout(hidden[-1])
        out = self.fc(hidden)
        return self.sigmoid(out)





In [1164]:

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import csv
input_dim = 300  # 你的Word2Vec向量维度
hidden_dim = 256  # LSTM隐藏层维度，可以根据需要调整
output_dim = 1  # 输出维度，因为是二进制分类，所以是1

model = RNNModel(input_dim, hidden_dim, output_dim)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-6)


In [1165]:
def train(model, iterator, optimizer, criterion):
    model.train()
    total_loss = 0
    total_samples = 0
    for batch in iterator:
        text, labels = batch
        optimizer.zero_grad()
        text = text.unsqueeze(1)

        predictions = model(text).squeeze(1)  # 使用squeeze消除不必要的维度
        loss = criterion(predictions, labels.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_samples += labels.size(0)  # 计算批次中的样本总数

    return total_loss / total_samples  # 使用总样本数来计算平均损失

def evaluate(model, iterator, criterion):
    model.eval()
    total_loss = 0
    total_samples = 0
    with torch.no_grad():
        for batch in iterator:
            text, labels = batch  
            text = text.unsqueeze(1)      
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, labels.float())
            total_loss += loss.item()
            total_samples += labels.size(0)  # 计算批次中的样本总数

    return total_loss / total_samples  # 使用总样本数来计算平均损失



N_EPOCHS = 10
for epoch in range(N_EPOCHS):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_loss = evaluate(model, test_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tTest Loss: {test_loss:.3f}')
    best_test_loss = float('inf')
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'model_weights.pth')




Epoch: 01
	Train Loss: 0.006
	Test Loss: 0.014
Epoch: 02
	Train Loss: 0.006
	Test Loss: 0.016
Epoch: 03
	Train Loss: 0.006
	Test Loss: 0.016
Epoch: 04
	Train Loss: 0.005
	Test Loss: 0.017
Epoch: 05
	Train Loss: 0.005
	Test Loss: 0.017
Epoch: 06
	Train Loss: 0.005
	Test Loss: 0.018
Epoch: 07
	Train Loss: 0.005
	Test Loss: 0.018
Epoch: 08
	Train Loss: 0.005
	Test Loss: 0.019
Epoch: 09
	Train Loss: 0.005
	Test Loss: 0.021
Epoch: 10
	Train Loss: 0.005
	Test Loss: 0.023


In [1166]:

print(model)


RNNModel(
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (sigmoid): Sigmoid()
)


In [1167]:
print(model.state_dict())


OrderedDict([('lstm.weight_ih_l0', tensor([[ 0.2453,  0.0603,  0.2121,  ...,  0.0303,  0.0552, -0.2619],
        [ 0.1133, -0.0146,  0.0457,  ..., -0.1483, -0.0580, -0.2832],
        [ 0.0525, -0.0652,  0.0851,  ...,  0.0359,  0.0412, -0.0184],
        ...,
        [ 0.2066, -0.1242,  0.2158,  ..., -0.1076,  0.0676, -0.2079],
        [-0.0741,  0.1407,  0.1275,  ..., -0.1842,  0.0337,  0.1243],
        [ 0.0140, -0.0838,  0.0657,  ...,  0.1187,  0.0636,  0.0277]])), ('lstm.weight_hh_l0', tensor([[-2.3338e-37,  5.3347e-38, -2.8043e-37,  ..., -6.5526e-37,
          1.4102e-37,  4.4913e-37],
        [-2.4571e-37,  5.4040e-37,  1.1579e-39,  ...,  5.2356e-38,
         -6.0960e-37,  9.3410e-38],
        [-1.5984e-37, -6.7281e-37, -3.6514e-37,  ...,  2.2764e-37,
         -8.8775e-38,  3.6292e-38],
        ...,
        [-2.3577e-37, -5.2705e-37,  8.1974e-39,  ..., -3.4465e-37,
          5.0054e-37,  2.5038e-37],
        [ 5.0215e-38, -1.6185e-37, -1.3887e-37,  ...,  6.8961e-37,
         -7.060

In [1168]:
X2 =test_set['text'].tolist()
max_length2 = max([len(seq) for seq in X2])
word2vec2 = Word2Vec(X2, vector_size=300, window=10, min_count=1, workers=4)
test_set = np.array([get_text_vector(text, word2vec2) for text in X2])
test_set_tensor = torch.tensor(test_set, dtype=torch.float32).to('cuda' if torch.cuda.is_available() else 'cpu')

In [1169]:
test_set_tensor

tensor([[-0.0224,  0.4832,  0.1261,  ...,  0.1071,  0.2194, -0.1277],
        [-0.0863,  0.3296, -0.0026,  ...,  0.0606,  0.1639, -0.2249],
        [-0.0082,  0.4067,  0.1255,  ...,  0.0950,  0.1805, -0.0655],
        ...,
        [ 0.0097,  0.4410,  0.1632,  ...,  0.1038,  0.1873, -0.0308],
        [-0.0147,  0.5067,  0.1441,  ...,  0.1138,  0.2276, -0.1135],
        [-0.0473,  0.4123,  0.0657,  ...,  0.0882,  0.1994, -0.1681]])

In [1170]:

model.load_state_dict(torch.load('model_weights.pth'))

# 设置模型为评估模式
model.eval()

# 假设test_data是你的测试数据，是一个列表，每个元素都是数字化的句子


import pandas as pd

# 假设test_data是你的测试数据，是一个Pandas DataFrame，包含一个名为'text'的列
predictions = []  # 用于存储预测结果的列表



with torch.no_grad():
    for index, text in enumerate(test_set_tensor):
        text = text.view(-1, 1, 300)
        predictions_tensor = model(text)
        predicted_class = (predictions_tensor > 0.901).float() 
        label = int(predicted_class.item())         
        predictions.append({"id": index, "label": label})


In [1171]:

model.load_state_dict(torch.load('model_weights.pth'))

# 设置模型为评估模式
model.eval()

# 假设test_data是你的测试数据，是一个列表，每个元素都是数字化的句子


import pandas as pd

# 假设test_data是你的测试数据，是一个Pandas DataFrame，包含一个名为'text'的列
values = []  # 用于存储预测结果的列表



with torch.no_grad():
    for index, text in enumerate(test_set_tensor):
        text = text.view(-1, 1, 300)
        predictions_tensor = model(text)
        values.append(predictions_tensor)



In [1172]:
tensor_list = [tensor.item() for tensor in values]
numpy_array = np.array(tensor_list)

median = np.median(numpy_array)

print("median:", median)


median: 0.8506362438201904


In [1173]:
predictions[:10]

[{'id': 0, 'label': 0},
 {'id': 1, 'label': 1},
 {'id': 2, 'label': 0},
 {'id': 3, 'label': 0},
 {'id': 4, 'label': 0},
 {'id': 5, 'label': 0},
 {'id': 6, 'label': 0},
 {'id': 7, 'label': 0},
 {'id': 8, 'label': 1},
 {'id': 9, 'label': 0}]

In [1174]:
import csv
output_csv_file = 'predictions.csv'
with open(output_csv_file, 'w', newline='') as csvfile:
    fieldnames = ['id', 'class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for prediction in predictions:
        writer.writerow({'id': prediction['id'], 'class': prediction['label']})

print(f'Predictions saved to {output_csv_file}')

Predictions saved to predictions.csv


In [1175]:
result = pd.read_csv('predictions.csv')

In [1176]:
result[result['class'] == 1]

,id,class
1,1,1
8,8,1
13,13,1
14,14,1
15,15,1
...,...,...
983,983,1
986,986,1
987,987,1
990,990,1


In [1177]:
result.to_csv('rnn_word2.csv', index=False)